# Set environment

In [1]:
### basic tools
import numpy as np
import pandas as pd

### show or plot
from IPython.display import display, HTML

### database
import sqlite3

### directory
datadir = "./data/"

### helper function
head = lambda x, n = 6: x[:n]
tail = lambda x, n = 6: x[-n:]

# Working with Relational Databases and SQL

**1**. 75 points

Convert the flat file data in `data/flat.csv` into a well-structured relational database in SQLite3 stored as `data/faculty.db`. Note - salary information is confidential and should be kept in a separate table from other personal data.

-----

## Import data

take a look at the csv file

In [2]:
flat = pd.read_csv(datadir + 'flat.csv', keep_default_na = False)

##### display the dimension and part of the DataFrame
print(flat.shape)
flat.sample(10)

(1523, 14)


,name,gender,age,height,weight,salary,nationality,code,country,language1,language2,language3,first,last
343,Deonna Stanley,Female,41,1.69,58,90000,Bolivian,BO,Bolivia,PL-SQL,,,Deonna,Stanley
644,James Morin,Male,35,1.89,38,63000,Italian,IT,Italy,,,,James,Morin
431,Emerson Richard,Male,36,1.85,48,158000,Taiwanese,TW,Taiwan,,,,Emerson,Richard
894,Loura Morrison,Female,16,1.86,55,122000,Venezuelan,VE,Venezuela,Clojure,CoffeeScript,,Loura,Morrison
755,Keeley Reese,Female,20,1.98,68,174000,Australian,AU,Australia,,,,Keeley,Reese
817,Larisa Hardin,Female,42,1.87,54,96000,Irish,IE,Ireland,,,,Larisa,Hardin
436,Emory Rutledge,Male,66,1.73,64,177000,Venezuelan,VE,Venezuela,,,,Emory,Rutledge
1500,Yen Wilder,Female,29,1.72,75,70000,British,GB,United Kingdom,PowerShell,,,Yen,Wilder
1367,Tandy Frederick,Female,40,1.98,58,118000,Brazilian,BR,Brazil,Dylan,Dylan,,Tandy,Frederick
73,Antone Harrington,Male,21,1.88,64,53000,Costa Rican,CR,Costa Rica,Lua,,,Antone,Harrington


There are 1523 people in the data.

In [3]:
flat.name.drop_duplicates().count()

1523

## Design tables

Here I am going to split the fkat DataFrame into the tables as follow

*Person*

| index | person_id| first | last      | age | height | weight | country_id | gender_id |
|-------|----------|-------|-----------|-----|--------|--------|------------|-----------|
|   0   |    0     | Aaron | Alexander | 54  |  1.7   |  90    |   GB       |     0     |
|   1   |    1     | Aaron | Kirby     | 59  |  1.69  |  43    |   SP       |     0     |
|   2   |    2     | Abram | Allen     | 41  |  1.7   |  44    |   IT       |     0     |


*Language*

|index | language_id | language_name |
|------|-------------|---------------|
|  0   |      1      |     PHP       |
|  1   |      2      |     Clojure   |
|  2   |      3      |     Dylan     |


*Person_Language*

| index | person_id | language_id |
|-------|-----------|-------------|
|   0   |     0     |     20      |
|   1   |    68     |     20      |
|   2   |    80     |     20      |



*Country*  

| index | country_id | country        | nationality |
|-------|------------|----------------|-------------|
|   0   |     GB     | United Kingdom | British     |
|   1   |     SP     | Spain          | Spanish     |
|   2   |     IT     | Italy          | Italian     |

*Confidential*

| index | person_id | salary |
|-------|-----------|--------|
|   0   |    0      | 151000 |
|   1   |    1      |  80000 |
|   2   |    2      |  75000 |

*Gender*

| index | gender_id | gender |
|-------|-----------|--------|
|   0   |     0     | Male   |
|   1   |     1     | Female | 



In [4]:
###########################################
### DataFrame for Person table
df_person_id = (flat
                [["name"]]
                .drop_duplicates()
                .reset_index(drop = True)
                .reset_index()
                .rename(index  = str, 
                        columns = {"index": "person_id"}))

# create flat with person id column
flat_pid = pd.merge(df_person_id, flat, on = 'name', how = 'inner')

###########################################
### DataFrame for Language table
df_lang         = (pd.melt(flat, 
                           value_vars = ['language1', 'language2', 'language3'], 
                           value_name = "language_name")
                   [["language_name"]]
                   .drop_duplicates()
                   .query("language_name != ''")
                   .reset_index(drop = True)
                   .reset_index()
                   .rename(index  = str, 
                           columns = {"index": "language_id"}))

###########################################
### DataFrame for Country table
df_country      = (flat
                   .loc[:, ["code", "country", "nationality"]]
                   .drop_duplicates()
                   .rename(index  = str, 
                           columns = {"code": "country_id"}))  

###########################################
### DataFrame for Confidential table, which contains the salary data only
df_confidential = (flat_pid
                   .loc[:, ["person_id", "salary"]]
                   .drop_duplicates()
                   .reset_index(drop = True)
                   .reset_index()
                   .rename(index  = str, 
                           columns = {"index": "confidential_id"}))  

###########################################
### DataFrame for Gender table
df_gender       = (flat
                   [["gender"]]
                   .drop_duplicates()
                   .reset_index(drop = True)
                   .reset_index()
                   .rename(index  = str, 
                           columns = {"index": "gender_id"}))  

##########################################
### DataFrame for Joint table connecting Person id and Language id
df = (pd.melt(flat_pid, 
              id_vars = ["person_id"],
              value_vars = ['language1', 'language2', 'language3'], 
              value_name = "language_name")
      .query("language_name != ''"))

df = pd.merge(df, df_lang, on = "language_name")
df = df.drop(columns = ['variable', 'language_name'])

df = (df
      .reset_index(drop = True)
      .reset_index()
      .rename(index  = str, 
              columns = {"index": "person_language_id"}))

df_person_lang = df

display all the tables and check if the results are what we expected

In [5]:
display(df_person_id.head(3))
display(df_lang.head(3))
display(df_country.head(3))
display(df_confidential.head(3))
display(df_gender.head(3))
display(df_person_lang.head(3))

,person_id,name
0,0,Aaron Alexander
1,1,Aaron Kirby
2,2,Abram Allen


,language_id,language_name
0,0,Haskell
1,1,Falcon
2,2,TypeScript


,country_id,country,nationality
0,GB,United Kingdom,British
1,SP,Spain,Spanish
2,IT,Italy,Italian


,confidential_id,person_id,salary
0,0,0,151000
1,1,1,80000
2,2,2,75000


,gender_id,gender
0,0,Male
1,1,Female


,person_language_id,person_id,language_id
0,0,0,0
1,1,162,0
2,2,179,0


## Country DataFrame: fixed duplicate country code

It turns out that the country table has several errors in two-letter country code, which is used for country id. First, I will find the errors by counting the occurence of each country id, which should be one.

In [6]:
### option 01
df_country.groupby("country_id").count().query("country != 1")
### option 02
# df_country.groupby("country_id").count().pipe(lambda df: df[df['country'] != 1])

,country,nationality
country_id,,
CH,2,2
CM,2,2


Get the country and nationality of the problem country codes

In [7]:
country_code_rep = df_country.groupby("country_id").count().query("country != 1").index.values
df_country.loc[lambda df: list(map(lambda x: x in country_code_rep, df["country_id"]))]

,country_id,country,nationality
5,CM,Cambodia,Cambodian
6,CM,Cameroon,Cameroonian
11,CH,China,Chinese
25,CH,Switzerland,Swiss


Based on the [ISO ALPHA-2 Code of country](https://www.nationsonline.org/oneworld/country_code_list.htm), the correct mapping of the country code and country should be

| Country     | code |
|-------------|------|
| Cambodia    |  KH  |
| Cameroon    |  CM  |
| China       |  CN  |
| Switzerland |  CH  |

Fix the country code for Cambodia and China

In [8]:
### change Cambodia CM -> KH and China CH -> CN
# get indices
df = df_country
idx1 = df.index[(df["country_id"] == "CM") & (df["country"] == "Cambodia")][0]
idx2 = df.index[(df["country_id"] == "CH") & (df["country"] == "China")][0]

# change country code
df = df_country.copy()
df.loc[idx1, "country_id"] = "KH"
df.loc[idx2, "country_id"] = "CN"
df_country_fix = df

# show that they are changed
country_name = ["Cambodia", "Cameroon", "China", "Switzerland"]
df.loc[lambda df: list(map(lambda x: x in country_name, df["country"]))]

,country_id,country,nationality
5,KH,Cambodia,Cambodian
6,CM,Cameroon,Cameroonian
11,CN,China,Chinese
25,CH,Switzerland,Swiss


## Set person data frame

Create the data frame for person table, which should contain person id, country id, and gender id.

In [9]:
df_person = (flat_pid
             .loc[:, [
                 "person_id", "first", "last", "age", "height", "weight", 
                 "country", "gender"]]
             .drop_duplicates())
                   
df_person = (pd.merge(df_person, df_gender,      on = "gender" )
             .drop(columns = ["gender"]))

df_person = (pd.merge(df_person, df_country_fix, on = "country")
             .drop(columns = ["country", "nationality"])
             .sort_values(by = "person_id"))

head(df_person)

,person_id,first,last,age,height,weight,gender_id,country_id
0,0,Aaron,Alexander,54,1.70,90,0,GB
39,1,Aaron,Kirby,59,1.69,43,0,SP
78,2,Abram,Allen,41,1.70,44,0,IT
79,3,Abram,Boyer,45,1.64,68,0,IT
658,4,Adaline,Barry,54,1.87,58,1,UY
1220,5,Adam,Lawrence,54,1.70,63,1,KH


## Create the tables in database

create well-structured relational database in SQLite3 stored as `data/faculty.db`

In [10]:
# Connecting to the database file
con = sqlite3.connect(datadir + "faculty.db")
cur = con.cursor()

Define helper function to create insert SQL query

In [11]:
def query_insert(table, cols, values):
    """Helper function to create insert query from table name (table), column names (cols), and observations (values)
    
    >>> query = query_insert("Person", ("x", "y"), [(1, 2, 'a'), (10, 20, 'b')])
    >>> print(query)
    INSERT INTO Person ('x', 'y')
    VALUES
    (1, 2, 'a'),
    (10, 20, 'b');
    """
    # initialize an insert query
    query = "INSERT INTO {table_name} {list_of_variables}\nVALUES".format(table_name = table, list_of_variables = cols)
    
    # add rows into query
    tmp = map(str, values)
    tmp = ",\n".join(tmp) + ";"
    
    return query + "\n" + tmp

### test the query_insert function
print(query_insert("Person", ("x", "y"), [(1, 2, 'a'), (10, 20, 'b')]))

INSERT INTO Person ('x', 'y')
VALUES
(1, 2, 'a'),
(10, 20, 'b');


**Tables that are going to be created in database**
```
df_lang
df_country
df_confidential
df_gender
df_person
df_person_lang
```

Declare the schema of all the tables, including setting up primary keys and foreign keys

In [12]:
cur.execute("DROP TABLE IF EXISTS Person;")
cur.execute("DROP TABLE IF EXISTS Language;")
cur.execute("DROP TABLE IF EXISTS Country;")
cur.execute("DROP TABLE IF EXISTS Confidential;")
cur.execute("DROP TABLE IF EXISTS Gender;")
cur.execute("DROP TABLE IF EXISTS Person_Language;")

#####################################################

query_create_lang = """
    CREATE TABLE Language (
        language_id INTEGER PRIMARY KEY,
        language_name varchar(255)
    );"""
cur.execute(query_create_lang)

#####################################################

query_create_country = """
    CREATE TABLE Country (
        country_id          varchar(2) PRIMARY KEY,
        country_name        varchar(255),
        country_nationality varchar(255)
    );"""
cur.execute(query_create_country)

#####################################################

query_create_gender = """
    CREATE TABLE Gender (
        gender_id   INTEGER PRIMARY KEY,
        gender_name varchar(255)
    );"""
cur.execute(query_create_gender)

#####################################################

query_create_person = """
    CREATE TABLE Person (
        person_id      INTEGER PRIMARY KEY,
        person_first   varchar(255),
        person_last    varchar(255),
        
        person_age     INTEGER,    
        person_height  FLOAT(3),
        person_weight  INTEGER,
        
        gender_id      INTEGER NOT NULL, 
        country_id     INTEGER NOT NULL,
        
        FOREIGN KEY (gender_id)  REFERENCES Gender  (gender_id),
        FOREIGN KEY (country_id) REFERENCES Country (country_id)
    );"""
cur.execute(query_create_person)

#####################################################

query_create_person_lang = """
    CREATE TABLE Person_Language (
        person_language_id  INTEGER PRIMARY KEY,
        person_id           INTEGER NOT NULL,
        language_id         INTEGER NOT NULL,  
        
        FOREIGN KEY (person_id)   REFERENCES Person   (person_id),
        FOREIGN KEY (language_id) REFERENCES Language (language_id)
    );"""
cur.execute(query_create_person_lang)

#####################################################

query_create_confidential = """
    CREATE TABLE Confidential (
        confidential_id      INTEGER PRIMARY KEY,
        person_id            INTEGER NOT NULL,
        confidential_salary  INTEGER,
        
        FOREIGN KEY (person_id) REFERENCES Person (person_id)
    );"""
cur.execute(query_create_confidential)

Check if the tables are created as expected

In [13]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('Language',), ('Country',), ('Gender',), ('Person',), ('Person_Language',), ('Confidential',)]


## Insert data values

In [14]:
###############################################################
### insert rows into table Language
query = query_insert(
    "Language", 
    ("language_id", "language_name"), 
    list(map(tuple, df_lang.values)))

cur.execute(query)

###############################################################
### insert rows into table Country
query = query_insert(
    "Country", 
    ("country_id", "country_name", "country_nationality"), 
    list(map(tuple, df_country_fix.values)))

cur.execute(query)

###############################################################
### insert rows into table Gender
query = query_insert(
    "Gender", 
    ("gender_id", "gender_name"), 
    list(map(tuple, df_gender.values)))

cur.execute(query)

###############################################################
### insert rows into table Person
query = query_insert(
    "Person", 
    ("person_id", "person_first", "person_last", 
     "person_age", "person_height", "person_weight", 
     "gender_id", "country_id"), 
    list(map(tuple, df_person.values)))

cur.execute(query)

###############################################################
### insert rows into table Person_Language (linker table)
query = query_insert(
    "Person_Language", 
    ("person_language_id", "person_id", "language_id"), 
    list(map(tuple, df_person_lang.values)))

cur.execute(query)

###############################################################
### insert rows into table Confidential
query = query_insert(
    "Confidential", 
    ("confidential_id", "person_id", "confidential_salary"), 
    list(map(tuple, df_confidential.values)))

cur.execute(query)

## Show all the tables

Check if all the tables are created in the faculty.db correctly

In [15]:
table_names = ["Language", "Country", "Gender", "Person", "Person_Language", "Confidential"]

for table_name in table_names:
    print("============" + table_name + "============")
    # query each table
    query = """SELECT * FROM {tn}""".format(tn = table_name)
    df = pd.read_sql_query(query, con = con)
    
    # display the query table
    display(head(df, n = 3))
    print("")

============Language============


,language_id,language_name
0,0,Haskell
1,1,Falcon
2,2,TypeScript



============Country============


,country_id,country_name,country_nationality
0,GB,United Kingdom,British
1,SP,Spain,Spanish
2,IT,Italy,Italian



============Gender============


...

,person_id,person_first,person_last,person_age,person_height,person_weight,gender_id,country_id
0,0,Aaron,Alexander,54,1.70,90,0,GB
1,1,Aaron,Kirby,59,1.69,43,0,SP
2,2,Abram,Allen,41,1.70,44,0,IT



============Person_Language============


,person_language_id,person_id,language_id
0,0,0,0
1,1,162,0
2,2,179,0



============Confidential============


,confidential_id,person_id,confidential_salary
0,0,0,151000
1,1,1,80000
2,2,2,75000


-----

**2**. 25 points

We want to find potential mentors for Abram	Boyer. Find all faculty members who know one or more of the same languages whose salary is at least $50,000 higher than his using SQL statements. Assume that the only information you have is that you need to find mentors meeting the criteria for the faculty member named `Abram Boyer`. In other words, the ONLY hard coded terms in your SQL query are `Abram` and `Boyer` and the salary differential.

You can use the `sql` magic extension or the `sqlite3` driver for this question.

## Check what languages Abram Boyer know and how much is Abram Boyer's salary

check languages Abram know

In [16]:
cur.execute("""DROP VIEW IF EXISTS language_view;""")

cur.execute("""
    CREATE VIEW language_view AS
    
    SELECT DISTINCT
        person_first  AS first, 
        person_last   AS last, 
        language_name AS language
    FROM Person, Language, Person_Language
    
    WHERE Person.person_first = "Abram" AND Person.person_last = "Boyer"
        AND Person.person_id = Person_Language.person_id
        AND Person_Language.language_id = Language.language_id
    ;""")

cur.execute("""SELECT * FROM language_view""")
cur.fetchall()

[('Abram', 'Boyer', 'Falcon'),
 ('Abram', 'Boyer', 'Lua'),
 ('Abram', 'Boyer', 'Io')]

See Abram Boyer's salary

In [17]:
cur.execute("""DROP VIEW IF EXISTS salary_view;""")

cur.execute("""
    CREATE VIEW salary_view AS
    SELECT 
        person_first AS first,
        person_last  AS last,
        confidential_salary AS salary
    FROM Person, Confidential 
    WHERE Person.person_first = "Abram" AND Person.person_last = "Boyer"
        AND Confidential.person_id = Person.person_id
    ;""")

cur.execute("""SELECT * FROM salary_view""")
cur.fetchall()

[('Abram', 'Boyer', 76000)]

## Find the person at least know one of the languages Abram Boyer know and the salary of the person is 50000 more than Abram Boyer

In [18]:
df = pd.read_sql_query("""
    SELECT DISTINCT
        person_first        AS first, 
        person_last         AS last, 
        confidential_salary AS salary
    FROM Confidential, Person, Language, Person_Language, language_view
    
    WHERE   Person.person_id = Confidential.person_id
        AND Person.person_id = Person_Language.person_id
        AND Person_Language.language_id = Language.language_id
        AND Language.language_name IN (SELECT language FROM language_view)
        AND salary >= (SELECT salary + 50000 from salary_view)
    ;""",
    con = con)

display(df)

,first,last,salary
0,Dawne,Knapp,185000
1,Dong,Walton,154000
2,Ellsworth,Jackson,155000
3,Everette,Garrison,126000
4,Guillermo,Herring,162000
5,Hai,Myers,130000
6,Jama,Ware,175000
7,Janessa,Chan,170000
8,Jarrod,Hall,143000
9,Jesenia,Wynn,145000
